In [ ]:
from pathlib import Path
import sys

# assume this notebook sits in project_root/notebooks
project_root = Path.cwd().parent  # adjust if needed
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from qepc.notebooks.notebook_env import init_notebook_env
from qepc.logging_utils import qstep

env = init_notebook_env()
cfg = env.config


In [ ]:
from qepc.nba.backtest_pipeline import run_nba_backtest
from qepc.core.strengths import TeamStrengthsConfig

strengths_cfg = TeamStrengthsConfig(
    lookback_seasons=3,
    min_games=20,
    recency_half_life_games=50,
)

results_df, metrics = run_nba_backtest(
    lookback_mode="all",   # or "years", "days"
    lookback_years=3,
    lookback_days=60,
    strengths_cfg=strengths_cfg,
)

print("📊 QEPC NBA Backtest – Summary")
for k, v in metrics.items():
    print(f"{k}: {v}")

results_df.head()
